##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Aşırı uyum (overfitting) ve yetersiz uyum (underfitting) nedir ?

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/overfit_and_underfit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Bu örneğimizde, 'tf.keras' API'sini kullanacağız. Tensorflow 'tf.keras' API'si ile ilgili daha fazla bilgiye [Keras guide](https://www.tensorflow.org/guide/keras) linki üzerinden ulaşabilirsiniz.

Önceki örneklerimizde (film yorumlarının sınıflandırılması, yakıt verimliliğinin tahminlenmesi) gördüğünüz gibi, doğrulama seti üzerinden ölçtüğümüz model doğruluğu, belirli bir epoch sayısında tepe noktasına ulaşıp, sonrasında azalmaya başlamaktadır. 

Diğer bir deyişle, modelimiz eğitim veri setine *aşırı uyumlu (overfit)* hale gelmektedir. Aşırı uyum ile nasıl baş edeceğimiz önemlidir. *eğitim veri seti* ile çok yüksek doğruluk değerine sahip olmamız mümkün olsa bile, asıl amacımız modelin daha önce görmediği veriler ile yani *test veri seti* ile yüksek doğrulukta sonuç üretebilmesi, bu şekilde genelleştirilmesidir.

Aşırı uyumun tersi *yetersiz uyum (underfittig)* 'dur. Yetersiz uyum, test verisi ile değerlendirdiğimizde modelde hala belirli bir iyileştirme alanının olması durumudur. Yetersiz uyum belirli nedenler sonucunda oluşur: Eğer model yeterince güçlü değil ise, çok fazla regülarize edildi ise, yeterince uzun süre eğitilmedi ise. Bu, ağın eğitim verisinden ilgili çıkarımları gerektiği kadar yapmadığı anlamına gelmektedir. 

Bunun yanında eğer modeli çok uzun süre eğitirseniz, moldel eğitim veri setine aşırı uyum sağlayarak, yaptığı çıkarımlar ile test veri setinde iyi şekilde çalışabilmesi için gerekli olan genelleme yeteneğini kaybedecektir. Modelimizin daha iyi performans göstermesi için dengeyi sağlamalıyız. Bu nedenle, aşağıda inceleyeceğimiz gibi, modelin uygun sayıda epoch ile nasıl eğitileceğini anlamak önemli bir beceridir. 

Aşırı uyumu engellemek için en iyi çözüm daha fazla eğitim verisinin kullanılmasıdır. Daha fazla veri ile eğitilen modeller doğal olarak daha iyi genelleme yeteneğine sahip olurlar. Bunun mümkün olmadığı durumlarda, ikinci en iyi çözüm *regularization* tekniklerini kullanmak olur. Bu teknikleri kullandığımızda, modelimizin sahip olacağı bilgi tipi ve miktarını sınırlarız. Model ağımız daha az miktarda çıkarımı hafızasına alırsa, optimizasyon süreci en önemli çıkarımları yapması için modeli zorlayacaktır. Bunun sonucunda modelin genelleme yeteneği artacaktır. 

Bu notebook'ta, IMBD film yorumlarının sınıflandırılması modelimizin iyileştirilmesi için en çok kullanılan iki regularizasyon tekniğini olan ağırlık regularizasyonu (weigth regularization) ve dropout tekniklerini kullanacaktır.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## IMDB veri setini indirelim

Önceki notebook'ta yer alan 'emmbeding' işlemi yerine, bu örnekte cümlelere 'multi-hot encoding' işlemi uygulayacağız. Modelimiz hızlı bir şekilde eğitim verisine aşırı uyum gösterecektir. Bu örnek ile aşırı uyumun nasıl oluştuğunu ve nasıl baş edeceğimizi göstermiş olacağız. 

'Multi-hot-encoding' ile listemizi 0'lar ve 1'ler içeren vektöre dönüştürelim. Örnek olarak bu işlem ile `[3, 5]` dizisi, 3 ve 5 indisleri 1 olan bunun dışındaki tüm değerleri 0 olan 10,000 boyutlu bir vektöre çevrilir. 

In [0]:
NUM_WORDS = 10000

(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=NUM_WORDS)

def multi_hot_sequences(sequences, dimension):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # set specific indices of results[i] to 1s
    return results


train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)

Sonuçta oluşan 'multi-hot' vektörlerden birine bakalım. Kelime indisleri frekanslarına göre sıralanmıştır, yani aşağıdaki grafikte görüldüğü gibi 0 indeksine yaklaştıkça daha fazla 1 değeri görürüz: 

In [0]:
plt.plot(train_data[0])

## Demonstrate overfitting

Aşırı uyumu engellemenin en basit yolu model boyutunun küçültülmesidir. Model boyutunu, modelin katman sayısına ve katmanları içersinde yere alan birim sayısına bağlı olan öğrenebilir parametrelerin sayısı belirler. Derin öğrenmede, modelde yer alan öğrenebilir parametrelerin sayısı modelin "kapasitesi" belirlemektedir. Daha basit haliyle, daha fazla parametreye sahip model, daha fazla "hafıza kapasitesine" sahiptir. Bu nedenle, model kusursuz bir şekilde eğitim verisine uyum sağlayacaktır. Genelleme yeteneğine sahip olmayan bu modelin, daha önce görmediği veriler ile başarılı bir çekilde tahminleme yapması mümkün değildir. 

Her zaman aklımızda tutmamız gerekir: derin öğrenme modelleri eğitim verisine uyum sağlama eğilimindedirler, asıl başarılması gereken bu uyum değil modele genelleme yeteneği kazandırabilmektir. 

Buna karşın, eğer ağın hafızaya alma kaynakları kısıtlı ise, öğrenmesi kolay olmayacaktır. Eğitim sürecinde kayıp değerini küçültmek ve daha fazla tahminleme gücü kazanmak için, modelin sıkıştırılmış çıkarımlar oluşturması gerekecektir. Bununla birlikte, modeli küçük oluşturusak eğitim veri setine uyum sağlamakta zorluk çekecektir.  "Çok fazla kapasite" ile "olması gerektiğinden az kapasite" arasında bir denge oluşturulması gerekmektedir.

Ne yazık ki, doğru büyüklükte bir model yapısını (modeldeki katman sayısı ve katmanların büyüklüğü) belirleyebileceğimiz sihirli bir formül bulunmamaktadır. Farklı model yapılarını deneyerek, problemimizin çözümü için en uygun modeli belirlememiz gerekmektedir. 

Uygun model boyutunu belirlemek için, göreceli olarak az katman ve parametreye sahip bir yapıyla başlamak en iyisidir. Doğrulama kayıp değerlerinde azalma görmemeye başladığımız zamana kadar, modelin katman sayısını veya katmanlar içerisindeki birimleri artırabiliriz. Film yorumlarının sınıflandırılmasını sağlayan ağımızda bunu deneyelim: 

Referans olarak, sadece '''Yoğun-Dense''' katmanlar içeren basit bir modelle başlayacağız. Sonrasında bu modelin daha küçük ve büyük versiyonlarını oluşturarak sonuçları karşılaştıracağız.

### Create a baseline model

In [0]:
baseline_model = keras.Sequential([
    # `input_shape` is only required here so that `.summary` works. 
    keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(NUM_WORDS,)),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

In [0]:
baseline_history = baseline_model.fit(train_data,
                                      train_labels,
                                      epochs=20,
                                      batch_size=512,
                                      validation_data=(test_data, test_labels),
                                      verbose=2)

### Create a smaller model

Let's create a model with less hidden units to compare against the baseline model that we just created:

In [0]:
smaller_model = keras.Sequential([
    keras.layers.Dense(4, activation=tf.nn.relu, input_shape=(NUM_WORDS,)),
    keras.layers.Dense(4, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

smaller_model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

And train the model using the same data:

In [0]:
smaller_history = smaller_model.fit(train_data,
                                    train_labels,
                                    epochs=20,
                                    batch_size=512,
                                    validation_data=(test_data, test_labels),
                                    verbose=2)

### Create a bigger model

As an exercise, you can create an even larger model, and see how quickly it begins overfitting.  Next, let's add to this benchmark a network that has much more capacity, far more than the problem would warrant:

In [0]:
bigger_model = keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(NUM_WORDS,)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

bigger_model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy','binary_crossentropy'])

bigger_model.summary()

And, again, train the model using the same data:

In [0]:
bigger_history = bigger_model.fit(train_data, train_labels,
                                  epochs=20,
                                  batch_size=512,
                                  validation_data=(test_data, test_labels),
                                  verbose=2)

### Plot the training and validation loss

<!--TODO(markdaoust): This should be a one-liner with tensorboard -->

The solid lines show the training loss, and the dashed lines show the validation loss (remember: a lower validation loss indicates a better model). Here, the smaller network begins overfitting later than the baseline model (after 6 epochs rather than 4) and its performance degrades much more slowly once it starts overfitting. 

In [0]:
def plot_history(histories, key='binary_crossentropy'):
  plt.figure(figsize=(16,10))
    
  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])


plot_history([('baseline', baseline_history),
              ('smaller', smaller_history),
              ('bigger', bigger_history)])

Notice that the larger network begins overfitting almost right away, after just one epoch, and overfits much more severely. The more capacity the network has, the quicker it will be able to model the training data (resulting in a low training loss), but the more susceptible it is to overfitting (resulting in a large difference between the training and validation loss).

## Strategies

### Add weight regularization



You may be familiar with Occam's Razor principle: given two explanations for something, the explanation most likely to be correct is the "simplest" one, the one that makes the least amount of assumptions. This also applies to the models learned by neural networks: given some training data and a network architecture, there are multiple sets of weights values (multiple models) that could explain the data, and simpler models are less likely to overfit than complex ones.

A "simple model" in this context is a model where the distribution of parameter values has less entropy (or a model with fewer parameters altogether, as we saw in the section above). Thus a common way to mitigate overfitting is to put constraints on the complexity of a network by forcing its weights only to take small values, which makes the distribution of weight values more "regular". This is called "weight regularization", and it is done by adding to the loss function of the network a cost associated with having large weights. This cost comes in two flavors:

* [L1 regularization](https://developers.google.com/machine-learning/glossary/#L1_regularization), where the cost added is proportional to the absolute value of the weights coefficients (i.e. to what is called the "L1 norm" of the weights).

* [L2 regularization](https://developers.google.com/machine-learning/glossary/#L2_regularization), where the cost added is proportional to the square of the value of the weights coefficients (i.e. to what is called the squared "L2 norm" of the weights). L2 regularization is also called weight decay in the context of neural networks. Don't let the different name confuse you: weight decay is mathematically the exact same as L2 regularization.

In `tf.keras`, weight regularization is added by passing weight regularizer instances to layers as keyword arguments. Let's add L2 weight regularization now.

In [0]:
l2_model = keras.models.Sequential([
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation=tf.nn.relu, input_shape=(NUM_WORDS,)),
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model_history = l2_model.fit(train_data, train_labels,
                                epochs=20,
                                batch_size=512,
                                validation_data=(test_data, test_labels),
                                verbose=2)

```l2(0.001)``` means that every coefficient in the weight matrix of the layer will add ```0.001 * weight_coefficient_value**2``` to the total loss of the network. Note that because this penalty is only added at training time, the loss for this network will be much higher at training than at test time.

Here's the impact of our L2 regularization penalty:

In [0]:
plot_history([('baseline', baseline_history),
              ('l2', l2_model_history)])

As you can see, the L2 regularized model has become much more resistant to overfitting than the baseline model, even though both models have the same number of parameters.

### Add dropout

Dropout is one of the most effective and most commonly used regularization techniques for neural networks, developed by Hinton and his students at the University of Toronto. Dropout, applied to a layer, consists of randomly "dropping out" (i.e. set to zero) a number of output features of the layer during training. Let's say a given layer would normally have returned a vector [0.2, 0.5, 1.3, 0.8, 1.1] for a given input sample during training; after applying dropout, this vector will have a few zero entries distributed at random, e.g. [0, 0.5, 
1.3, 0, 1.1]. The "dropout rate" is the fraction of the features that are being zeroed-out; it is usually set between 0.2 and 0.5. At test time, no units are dropped out, and instead the layer's output values are scaled down by a factor equal to the dropout rate, so as to balance for the fact that more units are active than at training time.

In tf.keras you can introduce dropout in a network via the Dropout layer, which gets applied to the output of layer right before.

Let's add two Dropout layers in our IMDB network to see how well they do at reducing overfitting:

In [0]:
dpt_model = keras.models.Sequential([
    keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(NUM_WORDS,)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

dpt_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy','binary_crossentropy'])

dpt_model_history = dpt_model.fit(train_data, train_labels,
                                  epochs=20,
                                  batch_size=512,
                                  validation_data=(test_data, test_labels),
                                  verbose=2)

In [0]:
plot_history([('baseline', baseline_history),
              ('dropout', dpt_model_history)])

Adding dropout is a clear improvement over the baseline model. 


To recap: here the most common ways to prevent overfitting in neural networks:

* Get more training data.
* Reduce the capacity of the network.
* Add weight regularization.
* Add dropout.

And two important approaches not covered in this guide are data-augmentation and batch normalization.